# 使用PyTorch计算梯度数值

**PyTorch的Autograd模块实现了深度学习的算法中的向传播求导数**，在张量（Tensor类）上的所有操作，**Autograd都能为他们自动提供微分**，简化了手动计算导数的复杂过程。

在0.4以前的版本中，Pytorch使用Variable类来自动计算所有的梯度Variable类主要包含三个属性：
data：保存Variable所包含的Tensor；grad：保存data对应的梯度，grad也是个Variable，而不是Tensor，它和data的形状一样；grad_fn：指向一个Function对象，这个Function用来反向传播计算输入的梯度。

从0.4起， Variable 正式合并入Tensor类，通过Variable嵌套实现的自动微分功能已经整合进入了Tensor类中。虽然为了代码的兼容性还是可以使用Variable(tensor)这种方式进行嵌套，但是这个操作其实什么都没做。

所以，以后的代码建议直接使用Tensor类进行操作，因为官方文档中已经将Variable设置成过期模块。

要想通过Tensor类本身就支持了使用autograd功能，只需要设置.requries_grad=True

Variable类中的的grad和grad_fn属性已经整合进入了Tensor类中

## Autograd

在张量创建时，通过设置 requires_grad 标识为Ture来告诉Pytorch需要对该张量进行自动求导，PyTorch会记录该张量的每一步操作历史并自动计算

In [2]:
import torch
torch.__version__

'1.4.0+cpu'

In [3]:
x = torch.rand(5, 5, requires_grad=True)
x

tensor([[0.2025, 0.3194, 0.8941, 0.6335, 0.5591],
        [0.8761, 0.8769, 0.3959, 0.7028, 0.2674],
        [0.5021, 0.8767, 0.0588, 0.1758, 0.7495],
        [0.0939, 0.3937, 0.8548, 0.2848, 0.1980],
        [0.8201, 0.0061, 0.9648, 0.6748, 0.1585]], requires_grad=True)

In [4]:
y = torch.rand(5, 5, requires_grad=True)
y

tensor([[0.4860, 0.2934, 0.8379, 0.1004, 0.4380],
        [0.8185, 0.3462, 0.6865, 0.2399, 0.7454],
        [0.6343, 0.9569, 0.4389, 0.0977, 0.8367],
        [0.0446, 0.7319, 0.9025, 0.6254, 0.4397],
        [0.0015, 0.3773, 0.2392, 0.0943, 0.0668]], requires_grad=True)

In [5]:
z=torch.sum(x+y)
z

tensor(24.0200, grad_fn=<SumBackward0>)

在张量进行操作后，grad_fn已经被赋予了一个新的函数，这个函数引用了一个创建了这个Tensor类的Function对象。
Tensor和Function互相连接生成了一个非循环图，它记录并且编码了完整的计算历史。每个张量都有一个.grad_fn属性，如果这个张量是用户手动创建的那么这个张量的grad_fn是None。

下面我们来调用反向传播函数，计算其梯度

## 简单的自动求导